# Chapter 5 : Creating Webpages with Controllers in Django: Views and URL configurations

A webpage consists of 
1. The data contained in the webpage
2. The URL (location) of the webpage. 

Django follows this abstraction by splitting the Controller into two parts. **Django views** give Django the data of the webpage. The URL associated with each view is listed in the **URL configuration**.

## Django Views
The view is where webpage data is generated. The developer uses the view to interact with the database, load and render the template, and perform any other logic necessary to displaying a webpage.
A Django view is any **Python callable** (function, class, or object) that meets the following two requirements:
   * Accepts an HttpRequest object as argument
   * Returns an HttpResponse object
   
An `HttpRequest` object contains all of the information about the page requested, any data the user is passing to the website, and any data the browser is sending about the user. The `HttpResponse` returns an HTTP code as well as any data the developer chooses to return to the user.

Any view that is built using a function is called a *function view*, and any view that is an object is called a *class-based view*

### Example of a view function in views.py
```python
from django.http.response import HttpResponse
from django.template import loader
from .models import model_class_name

def homepage(request):
    data = model_class_name.objects.all()
    template = loader.get_template('app_name/file.html')
    context = { 'data': data }
    output = template.render(context)
    return HttpResponse(output)
```

## Django's Use of Views and URL Configurations

### Usage of regular expression in urls.py

Regular expression can be used to recognise URL patterns to match. Django make use of **named group** in regular expression to extract the pattern string into a variable group, that can be passes as a parameter list into a view function. Python regular expressions identify named groups with the text `(?P<name>pattern)` , where name is the name of the group and pattern is the actual regular expression pattern. In a URL pattern, Django takes any named group and passes its value to the view the URL pattern points to. Consider the example

**urls.py**
```python
from django.urls import re_path
from app_name.views import homepage

urlpatterns = [
    re_path(r'^super/(?P<slug>[\w\-]+)/\$', hompage),
]
```
The list matches the url of pattern `super/some_slug_string/`. The  regular express `[\w\-]+` tells to match any combinations of alphanumerical string, underscore ( _ ) and dash (-) symbols. The recognized symbols are saved into a name called `slug`. The `^` marks the begining of the URL and `$` marks the end of the URL pattern. 

The view function `homepage` is passed with a parameter `slug`. Hence the function can accept the parameter and the definition would look like

**app_name/views.py**
```python
from django.http.response import HttpResponse
from django.template import loader
from .models import model_class_name

def homepage(request, slug):
    # slug variable can be used here!
    return HttpResponse()
```      

## Handling Errors : returning 404

Update the view function to handle or return 404 object

**method one : Raising exception**
```python
from django.http.response import Http404, HttpResponse

def my_view(request):
    try:
        # error prone code goes here!
    except Exception as e : #catch the exception
        raise Http404  # raise 404 exception
    
    #some more code
    return HttpResponse(output)
```
---
**method two : Returning 404 object**
```python
from django.http.response import HttpResponse, HttpResponseNotFound

def my_view(request):
    try:
        # error prone code goes here!
    except Exception as e : #catch the exception
        return HttpResponceNotFound("<h2>No such resource</h2>")
        # returns an http response which inturn raises 404 exception
    
    #some more code
    return HttpResponse(output)
```

## Django shortcuts
Most of the time, there are a lot of identical task like loading the template, fetching data from database, generate 404 if data is not found, return and http responce etc. These functionaly are combined together using various functions defined in `django.shortcuts`

`get_object_or_404` : The shortcut expects to have the model class and the desired query passed as arguments and will return the object if it finds one. If not, it raises Http404.

`render` : It replaces the common task like Load a template file as a Template object, Render the Template with the Context/dictionary, Instantiate an HttpResponse object with the rendered result.

```python
from django.shortcuts import get_object_or_404, render
def my_view(request,slug):
    user_info = get_object_or_404(UserDetails, slug__iexact=slug)
    context = {'user':user_info}
    return render(request, 'app_name/user_detail.html',context)
```
Thus a simple page is processed with just few lines!

## Class based views
Any Python callable that accepts an HttpRequest object as argument and returns an HttpResponse object is deemed a view in Django. Django introduced a class to create view objects. The class itself is simply called View, and developers refer to classes that inherit View as **class-based views (CBVs)**. These classes behave exactly like function views but come with several unexpected benefits.

**app_name/views.py**
```python
from django.views.generic import View

class BlogList(View):
    
    def function(self,request):
        #some code
        return render(request, 'file.html', context)
```

The import of View typically causes confusion because it implies that View is generic, leading people to confuse View and class-based views with *generic class-based views (GCBVs)*. **GCBVs are not the same as CBVs**

A CBV is simply a class that inherits View and meets the basic requirements of being a Django view: a view is a Python callable that always accepts an HttpRequest object and always returns an HttpResponse object.


The CBV organizes view behavior for a URI or set of URIs **according to HTTP methods**. Specifically, View is built such that it expects us to define any of the following: get() , post() , put() , patch() , delete() , trace() . 

In the event that the method is undefined, the CBV raises an HTTP 405 error. At first glance, CBVs are far more complex than function views. However, CBVs are more clearly organized, allow for shared behavior according to OOP, and better adhere to the rules of HTTP out of the box.